# 🎓 Relatório do Otimizador de Grade Horária

Este notebook documenta, valida e executa a aplicação web de otimização de grades curriculares usando Programação Linear Inteira Mista (MILP).

## 📋 Índice

1. [🔧 Configuração Automática do Ambiente](#configuracao-ambiente) ⭐ **EXECUTE PRIMEIRO**
2. [Visão Geral do Projeto](#visao-geral)
3. [Validação do Ambiente](#validacao-ambiente)
4. [Validação dos Dados](#validacao-dados)
5. [Análise dos Dados](#analise-dados)
6. [Testes do Sistema](#testes-sistema)
7. [Iniciar Aplicação Web](#iniciar-app)
8. [Status e Monitoramento](#status-monitoramento)

---

### 🚀 Como Usar

**Opção 1: Executar Tudo Automaticamente (Recomendado)**
- Clique em **"Run All"** ou **"Cell > Run All"**
- O notebook criará o venv, instalará dependências e executará tudo automaticamente

**Opção 2: Executar Célula por Célula**
- Execute as células em ordem
- A primeira seção configura o ambiente automaticamente

---


## 🔧 Configuração Automática do Ambiente {#configuracao-ambiente}

Esta seção configura automaticamente o ambiente virtual (venv) e instala todas as dependências necessárias.

**Execute esta seção primeiro!** Ela será executada automaticamente ao usar "Run All".


In [1]:
import sys
import subprocess
import os
from pathlib import Path
import platform

print("=" * 70)
print("🔧 CONFIGURAÇÃO AUTOMÁTICA DO AMBIENTE")
print("=" * 70)

# Diretório atual
current_dir = Path.cwd()
venv_dir = current_dir / "venv"
venv_python = None

# Detectar sistema operacional
is_windows = platform.system() == "Windows"

if is_windows:
    venv_python = venv_dir / "Scripts" / "python.exe"
    venv_pip = venv_dir / "Scripts" / "pip.exe"
else:
    venv_python = venv_dir / "bin" / "python"
    venv_pip = venv_dir / "bin" / "pip"

print(f"\n📁 Diretório do projeto: {current_dir}")
print(f"🐍 Python atual: {sys.executable}")
print(f"💻 Sistema operacional: {platform.system()}")

# Verificar se venv já existe
venv_exists = venv_dir.exists() and venv_python.exists()

if venv_exists:
    print(f"\n✅ Ambiente virtual já existe em: {venv_dir}")
    print(f"   Python do venv: {venv_python}")
else:
    print(f"\n📦 Criando ambiente virtual em: {venv_dir}")
    try:
        # Criar venv
        subprocess.run(
            [sys.executable, "-m", "venv", str(venv_dir)],
            check=True,
            capture_output=True,
            text=True
        )
        print("✅ Ambiente virtual criado com sucesso!")
    except subprocess.CalledProcessError as e:
        print(f"❌ Erro ao criar ambiente virtual: {e}")
        print(f"   Saída: {e.stdout}")
        print(f"   Erros: {e.stderr}")
        raise


🔧 CONFIGURAÇÃO AUTOMÁTICA DO AMBIENTE

📁 Diretório do projeto: d:\Arquivos\Desktop\otm
🐍 Python atual: c:\Users\Hugo\AppData\Local\Programs\Python\Python312\python.exe
💻 Sistema operacional: Windows

📦 Criando ambiente virtual em: d:\Arquivos\Desktop\otm\venv
✅ Ambiente virtual criado com sucesso!


In [2]:
# Verificar se requirements.txt existe
requirements_file = current_dir / "requirements.txt"

if not requirements_file.exists():
    print(f"\n❌ Arquivo requirements.txt não encontrado em: {requirements_file}")
    raise FileNotFoundError(f"requirements.txt não encontrado em {requirements_file}")

print(f"\n📋 Arquivo de dependências encontrado: {requirements_file}")

# Ler requirements.txt para mostrar quantas dependências serão instaladas
with open(requirements_file, 'r', encoding='utf-8') as f:
    requirements_lines = [line.strip() for line in f if line.strip() and not line.strip().startswith('#')]
    
print(f"   Total de pacotes a instalar: {len(requirements_lines)}")

# Verificar se pip precisa ser atualizado
print(f"\n🔄 Atualizando pip no ambiente virtual...")
try:
    result = subprocess.run(
        [str(venv_pip), "install", "--upgrade", "pip"],
        check=True,
        capture_output=True,
        text=True,
        timeout=60
    )
    print("✅ pip atualizado com sucesso!")
except subprocess.CalledProcessError as e:
    print(f"⚠️ Aviso ao atualizar pip: {e.stderr}")
except subprocess.TimeoutExpired:
    print("⚠️ Timeout ao atualizar pip (continuando mesmo assim...)")
except Exception as e:
    print(f"⚠️ Erro ao atualizar pip: {e} (continuando mesmo assim...)")



📋 Arquivo de dependências encontrado: d:\Arquivos\Desktop\otm\requirements.txt
   Total de pacotes a instalar: 45

🔄 Atualizando pip no ambiente virtual...
⚠️ Aviso ao atualizar pip: ERROR: To modify pip, please run the following command:
d:\Arquivos\Desktop\otm\venv\Scripts\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: d:\Arquivos\Desktop\otm\venv\Scripts\python.exe -m pip install --upgrade pip



In [3]:
# Instalar dependências
print(f"\n📦 Instalando dependências do requirements.txt...")
print("   Isso pode levar alguns minutos na primeira execução...")

try:
    result = subprocess.run(
        [str(venv_pip), "install", "-r", str(requirements_file)],
        check=True,
        capture_output=True,
        text=True,
        timeout=600  # 10 minutos de timeout
    )
    
    # Contar pacotes instalados (aproximado)
    installed_count = result.stdout.count("Successfully installed")
    if installed_count > 0:
        print(f"✅ {installed_count} pacote(s) instalado(s) com sucesso!")
    else:
        print("✅ Dependências instaladas (ou já estavam instaladas)!")
        
except subprocess.CalledProcessError as e:
    print(f"\n❌ Erro ao instalar dependências:")
    print(f"   Código de saída: {e.returncode}")
    if e.stdout:
        # Mostrar últimas linhas da saída
        lines = e.stdout.split('\n')
        print(f"   Últimas linhas da saída:")
        for line in lines[-10:]:
            if line.strip():
                print(f"     {line}")
    if e.stderr:
        # Mostrar erros
        error_lines = e.stderr.split('\n')
        print(f"   Erros:")
        for line in error_lines[-10:]:
            if line.strip():
                print(f"     {line}")
    raise
    
except subprocess.TimeoutExpired:
    print(f"\n❌ Timeout ao instalar dependências (limite de 10 minutos excedido)")
    print("   Tente executar manualmente: pip install -r requirements.txt")
    raise
    
except Exception as e:
    print(f"\n❌ Erro inesperado: {e}")
    raise



📦 Instalando dependências do requirements.txt...
   Isso pode levar alguns minutos na primeira execução...
✅ 1 pacote(s) instalado(s) com sucesso!


In [4]:
# Configurar sys.path para usar o venv
print(f"\n🔗 Configurando Python para usar o ambiente virtual...")

# Adicionar site-packages do venv ao sys.path
if is_windows:
    site_packages = venv_dir / "Lib" / "site-packages"
else:
    site_packages = venv_dir / "lib" / f"python{sys.version_info.major}.{sys.version_info.minor}" / "site-packages"

if site_packages.exists():
    if str(site_packages) not in sys.path:
        sys.path.insert(0, str(site_packages))
        print(f"✅ Adicionado ao sys.path: {site_packages}")
    else:
        print(f"✅ Já está no sys.path: {site_packages}")
else:
    print(f"⚠️ Site-packages não encontrado em: {site_packages}")
    print("   Tentando continuar mesmo assim...")

# Verificar se conseguimos importar pacotes principais do venv
print(f"\n🧪 Verificando instalação dos pacotes principais...")

test_packages = {
    "streamlit": "Framework web",
    "ortools": "Solver MILP",
    "pandas": "Manipulação de dados",
    "numpy": "Cálculos numéricos"
}

failed_imports = []
for package, description in test_packages.items():
    try:
        mod = __import__(package)
        version = getattr(mod, "__version__", "desconhecida")
        print(f"   ✅ {package:15s} {version:15s} - {description}")
    except ImportError:
        print(f"   ❌ {package:15s} {'NÃO INSTALADO':15s} - {description}")
        failed_imports.append(package)

if failed_imports:
    print(f"\n⚠️ ATENÇÃO: {len(failed_imports)} pacote(s) não puderam ser importados!")
    print("   Isso pode indicar que a instalação não foi completa.")
    print("   Tente executar manualmente:")
    print(f"   {venv_pip} install {' '.join(failed_imports)}")
else:
    print(f"\n✅ Todos os pacotes principais estão disponíveis!")
    print(f"\n🎉 Ambiente configurado com sucesso!")
    print(f"   Python do venv: {venv_python}")
    print(f"   Você pode continuar executando as células seguintes.")



🔗 Configurando Python para usar o ambiente virtual...
✅ Adicionado ao sys.path: d:\Arquivos\Desktop\otm\venv\Lib\site-packages

🧪 Verificando instalação dos pacotes principais...
   ✅ streamlit       1.51.0          - Framework web
   ✅ ortools         9.14.6206       - Solver MILP
   ✅ pandas          2.3.2           - Manipulação de dados
   ✅ numpy           2.3.3           - Cálculos numéricos

✅ Todos os pacotes principais estão disponíveis!

🎉 Ambiente configurado com sucesso!
   Python do venv: d:\Arquivos\Desktop\otm\venv\Scripts\python.exe
   Você pode continuar executando as células seguintes.


---

**Nota:** Se você quiser usar este ambiente virtual em um novo kernel Jupyter, execute:

```bash
# No terminal, ative o venv e instale ipykernel:
# Windows:
venv\Scripts\activate
python -m ipykernel install --user --name=otm-venv --display-name "Python (otm-venv)"

# Linux/macOS:
source venv/bin/activate
python -m ipykernel install --user --name=otm-venv --display-name "Python (otm-venv)"
```

Depois, selecione o kernel "Python (otm-venv)" no Jupyter.

---


## 1. Visão Geral do Projeto {#visao-geral}

### Descrição

Aplicação web interativa para otimização de grades curriculares usando **Programação Linear Inteira Mista (MILP)** com o solver **Google OR-Tools (SCIP)**.

### Funcionalidades

- ✅ Seleção interativa de disciplinas concluídas
- ✅ Otimização automática de grade horária
- ✅ Minimização do tempo de graduação
- ✅ Respeito a pré-requisitos e restrições
- ✅ Visualização de resultados em tabela e grade semanal

### Tecnologias

- **Frontend:** Streamlit
- **Otimização:** Google OR-Tools (SCIP solver)
- **Linguagem:** Python 3.8+
- **Estrutura:** Arquitetura modular (models, services, utils, ui)


## 2. Validação do Ambiente {#validacao-ambiente}

Verificando Python, dependências e estrutura do projeto.


In [5]:
import sys
import subprocess
import importlib
from pathlib import Path
import json
import webbrowser
import time
from datetime import datetime
from typing import Dict, List, Optional

print("=" * 70)
print("🔍 VALIDAÇÃO DO AMBIENTE")
print("=" * 70)

# Versão do Python
print(f"\n✅ Python: {sys.version.split()[0]} ({sys.executable})")

# Verificar se estamos no diretório correto
current_dir = Path.cwd()
print(f"\n📁 Diretório atual: {current_dir}")

# Verificar estrutura do projeto
required_files = [
    "app.py",
    "config.py",
    "requirements.txt",
    "models/disciplina.py",
    "services/data_loader.py",
    "services/optimizer.py",
    "utils/validators.py",
    "ui/components.py"
]

print("\n📋 Estrutura do projeto:")
missing_files = []
for file_path in required_files:
    full_path = current_dir / file_path
    if full_path.exists():
        print(f"  ✅ {file_path}")
    else:
        print(f"  ❌ {file_path} (NÃO ENCONTRADO)")
        missing_files.append(file_path)

if missing_files:
    print(f"\n⚠️ ATENÇÃO: {len(missing_files)} arquivo(s) não encontrado(s)!")
else:
    print("\n✅ Todos os arquivos necessários estão presentes!")


🔍 VALIDAÇÃO DO AMBIENTE

✅ Python: 3.12.4 (c:\Users\Hugo\AppData\Local\Programs\Python\Python312\python.exe)

📁 Diretório atual: d:\Arquivos\Desktop\otm

📋 Estrutura do projeto:
  ✅ app.py
  ✅ config.py
  ✅ requirements.txt
  ✅ models/disciplina.py
  ✅ services/data_loader.py
  ✅ services/optimizer.py
  ✅ utils/validators.py
  ✅ ui/components.py

✅ Todos os arquivos necessários estão presentes!


In [6]:
# Verificar dependências principais
print("\n" + "=" * 70)
print("📦 VERIFICAÇÃO DE DEPENDÊNCIAS")
print("=" * 70)

required_packages = {
    "streamlit": "Framework web",
    "ortools": "Solver MILP",
    "pandas": "Manipulação de dados",
    "numpy": "Cálculos numéricos"
}

missing_packages = []
for package, description in required_packages.items():
    try:
        mod = importlib.import_module(package)
        version = getattr(mod, "__version__", "desconhecida")
        print(f"  ✅ {package:15s} {version:15s} - {description}")
    except ImportError:
        print(f"  ❌ {package:15s} {'NÃO INSTALADO':15s} - {description}")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠️ ATENÇÃO: {len(missing_packages)} pacote(s) não instalado(s)!")
    print("\nPara instalar, execute:")
    print(f"  pip install {' '.join(missing_packages)}")
    print("\nOu instale todas as dependências:")
    print("  pip install -r requirements.txt")
else:
    print("\n✅ Todas as dependências principais estão instaladas!")



📦 VERIFICAÇÃO DE DEPENDÊNCIAS
  ✅ streamlit       1.51.0          - Framework web
  ✅ ortools         9.14.6206       - Solver MILP
  ✅ pandas          2.3.2           - Manipulação de dados
  ✅ numpy           2.3.3           - Cálculos numéricos

✅ Todas as dependências principais estão instaladas!


## 3. Validação dos Dados {#validacao-dados}

Verificando se os arquivos JSON de disciplinas e ofertas existem e são válidos.


In [7]:
print("\n" + "=" * 70)
print("📊 VALIDAÇÃO DOS DADOS")
print("=" * 70)

# Importar configuração
try:
    from config import config
    print(f"\n✅ Configuração carregada")
    print(f"   Base dir: {config.BASE_DIR}")
    print(f"   Web dir: {config.WEB_DIR}")
except Exception as e:
    print(f"\n❌ Erro ao carregar configuração: {e}")
    raise

# Validar caminhos
sucesso, mensagem_erro = config.validate_paths()
if not sucesso:
    print(f"\n❌ ERRO: {mensagem_erro}")
    raise FileNotFoundError(mensagem_erro)

print(f"\n✅ Arquivo de disciplinas: {config.DISCIPLINAS_PATH}")
print(f"   Existe: {config.DISCIPLINAS_PATH.exists()}")
print(f"   Tamanho: {config.DISCIPLINAS_PATH.stat().st_size / 1024:.1f} KB")

print(f"\n✅ Arquivo de ofertas: {config.OFERTAS_PATH}")
print(f"   Existe: {config.OFERTAS_PATH.exists()}")
print(f"   Tamanho: {config.OFERTAS_PATH.stat().st_size / 1024:.1f} KB")



📊 VALIDAÇÃO DOS DADOS

✅ Configuração carregada
   Base dir: d:\Arquivos\Desktop\otm
   Web dir: d:\Arquivos\Desktop\otm

✅ Arquivo de disciplinas: d:\Arquivos\Desktop\otm\attempt1\disciplinas.json
   Existe: True
   Tamanho: 27.9 KB

✅ Arquivo de ofertas: d:\Arquivos\Desktop\otm\attempt1\ofertas.json
   Existe: True
   Tamanho: 15.7 KB


In [8]:
# Carregar e validar JSONs
print("\n" + "=" * 70)
print("📖 CARREGANDO E VALIDANDO JSONs")
print("=" * 70)

try:
    with open(config.DISCIPLINAS_PATH, 'r', encoding='utf-8') as f:
        disciplinas_data = json.load(f)
    print(f"\n✅ Disciplinas carregadas: {len(disciplinas_data)} disciplinas")
    
    # Validar estrutura
    if disciplinas_data and isinstance(disciplinas_data, list):
        primeira = disciplinas_data[0]
        campos_esperados = ['id', 'nome', 'creditos', 'tipo']
        campos_presentes = [campo for campo in campos_esperados if campo in primeira]
        print(f"   Campos presentes: {', '.join(campos_presentes)}")
        if len(campos_presentes) == len(campos_esperados):
            print("   ✅ Estrutura válida")
        else:
            print(f"   ⚠️ Campos faltando: {set(campos_esperados) - set(campos_presentes)}")
    else:
        print("   ⚠️ Formato inesperado (esperado: lista de dicionários)")
        
except json.JSONDecodeError as e:
    print(f"\n❌ ERRO: JSON de disciplinas inválido: {e}")
    raise
except Exception as e:
    print(f"\n❌ ERRO ao carregar disciplinas: {e}")
    raise

try:
    with open(config.OFERTAS_PATH, 'r', encoding='utf-8') as f:
        ofertas_data = json.load(f)
    print(f"\n✅ Ofertas carregadas: {len(ofertas_data)} ofertas")
    
    # Validar estrutura
    if ofertas_data and isinstance(ofertas_data, list):
        primeira = ofertas_data[0]
        campos_esperados = ['disciplina_id', 'turma_id', 'horarios']
        campos_presentes = [campo for campo in campos_esperados if campo in primeira]
        print(f"   Campos presentes: {', '.join(campos_presentes)}")
        if len(campos_presentes) >= 2:
            print("   ✅ Estrutura válida")
        else:
            print(f"   ⚠️ Campos faltando: {set(campos_esperados) - set(campos_presentes)}")
    else:
        print("   ⚠️ Formato inesperado (esperado: lista de dicionários)")
        
except json.JSONDecodeError as e:
    print(f"\n❌ ERRO: JSON de ofertas inválido: {e}")
    raise
except Exception as e:
    print(f"\n❌ ERRO ao carregar ofertas: {e}")
    raise



📖 CARREGANDO E VALIDANDO JSONs

✅ Disciplinas carregadas: 159 disciplinas
   Campos presentes: id, nome, creditos, tipo
   ✅ Estrutura válida

✅ Ofertas carregadas: 105 ofertas
   Campos presentes: disciplina_id, turma_id
   ✅ Estrutura válida


## 4. Análise dos Dados {#analise-dados}

Analisando estatísticas dos dados carregados.


In [9]:
# Garantir que venv_python está disponível para iniciar o Streamlit
if 'venv_python' not in locals():
    current_dir = Path.cwd()
    venv_dir = current_dir / "venv"
    is_windows = platform.system() == "Windows"
    if is_windows:
        venv_python = venv_dir / "Scripts" / "python.exe"
    else:
        venv_python = venv_dir / "bin" / "python"
    
    if not venv_python.exists():
        print("\n❌ Ambiente virtual não encontrado!")
        print("   Execute primeiro as células de configuração do ambiente.")
        raise FileNotFoundError(f"Venv não encontrado: {venv_python}")
    print(f"✅ Python do venv configurado: {venv_python}")
else:
    print(f"✅ Python do venv já disponível: {venv_python}")


✅ Python do venv já disponível: d:\Arquivos\Desktop\otm\venv\Scripts\python.exe


In [10]:
print("\n" + "=" * 70)
print("📈 ANÁLISE DOS DADOS")
print("=" * 70)

# Análise de disciplinas
tipos_disciplinas = {}
total_creditos = 0
disciplinas_por_periodo = {}

for disc in disciplinas_data:
    # Contar por tipo
    tipo = disc.get('tipo', 'desconhecido')
    tipos_disciplinas[tipo] = tipos_disciplinas.get(tipo, 0) + 1
    
    # Somar créditos
    creditos = disc.get('creditos', 0)
    if isinstance(creditos, (int, float)):
        total_creditos += creditos
    
    # Contar por período sugerido
    periodo = disc.get('periodo_sugerido', 'N/A')
    if periodo != 'N/A' and periodo is not None:
        disciplinas_por_periodo[periodo] = disciplinas_por_periodo.get(periodo, 0) + 1

print(f"\n📚 DISCIPLINAS:")
print(f"   Total: {len(disciplinas_data)}")
print(f"   Total de créditos: {total_creditos}")
print(f"\n   Por tipo:")
for tipo, count in sorted(tipos_disciplinas.items()):
    print(f"     • {tipo:20s}: {count:3d} disciplina(s)")

if disciplinas_por_periodo:
    print(f"\n   Por período sugerido:")
    for periodo in sorted(disciplinas_por_periodo.keys()):
        print(f"     • Período {periodo:2s}: {disciplinas_por_periodo[periodo]:3d} disciplina(s)")



📈 ANÁLISE DOS DADOS

📚 DISCIPLINAS:
   Total: 159
   Total de créditos: 610.0

   Por tipo:
     • 10º Período         :   2 disciplina(s)
     • 1º Período          :   6 disciplina(s)
     • 2º Período          :   6 disciplina(s)
     • 3º Período          :   6 disciplina(s)
     • 4º Período          :   7 disciplina(s)
     • 5º Período          :   6 disciplina(s)
     • 6º Período          :   5 disciplina(s)
     • 7º Período          :   5 disciplina(s)
     • 8º Período          :   3 disciplina(s)
     • 9º Período          :   1 disciplina(s)
     • Disciplinas Optativas (Escolha Condicionada):  76 disciplina(s)
     • Disciplinas Optativas (Escolha Restrita):  36 disciplina(s)


In [11]:
# Análise de ofertas
disciplinas_com_oferta = set()
turmas_por_disciplina = {}
total_turmas = 0

for oferta in ofertas_data:
    disc_id = oferta.get('disciplina_id')
    turma_id = oferta.get('turma_id')
    
    if disc_id:
        disciplinas_com_oferta.add(disc_id)
        if disc_id not in turmas_por_disciplina:
            turmas_por_disciplina[disc_id] = set()
        if turma_id:
            turmas_por_disciplina[disc_id].add(turma_id)
            total_turmas += 1

print(f"\n📅 OFERTAS:")
print(f"   Total de ofertas: {len(ofertas_data)}")
print(f"   Total de turmas únicas: {total_turmas}")
print(f"   Disciplinas com oferta: {len(disciplinas_com_oferta)}")
print(f"   Disciplinas sem oferta: {len(disciplinas_data) - len(disciplinas_com_oferta)}")

# Estatísticas de turmas por disciplina
if turmas_por_disciplina:
    turmas_counts = [len(turmas) for turmas in turmas_por_disciplina.values()]
    print(f"\n   Estatísticas de turmas por disciplina:")
    print(f"     • Média: {sum(turmas_counts) / len(turmas_counts):.1f} turmas")
    print(f"     • Mínimo: {min(turmas_counts)} turmas")
    print(f"     • Máximo: {max(turmas_counts)} turmas")



📅 OFERTAS:
   Total de ofertas: 105
   Total de turmas únicas: 105
   Disciplinas com oferta: 61
   Disciplinas sem oferta: 98

   Estatísticas de turmas por disciplina:
     • Média: 1.7 turmas
     • Mínimo: 1 turmas
     • Máximo: 20 turmas


## 5. Testes do Sistema {#testes-sistema}

Executando testes de importação e validação dos módulos.


In [12]:
print("\n" + "=" * 70)
print("🧪 TESTES DO SISTEMA")
print("=" * 70)

test_results = []

# Teste 1: Config
try:
    from config import config
    test_results.append(("✅", "config.py", "OK"))
except Exception as e:
    test_results.append(("❌", "config.py", str(e)))

# Teste 2: Models
try:
    from models.grade import DadosDisciplinas, GradeResultado
    from models.disciplina import Disciplina
    test_results.append(("✅", "models/", "OK"))
except Exception as e:
    test_results.append(("❌", "models/", str(e)))

# Teste 3: Utils
try:
    from utils.validators import validate_semestre, validate_file_exists
    from utils.logging_config import setup_logging, get_logger
    test_results.append(("✅", "utils/", "OK"))
except Exception as e:
    test_results.append(("❌", "utils/", str(e)))

# Teste 4: Services - DataLoader
try:
    from services.data_loader import DataLoader, carregar_dados
    test_results.append(("✅", "services/data_loader.py", "OK"))
except Exception as e:
    test_results.append(("❌", "services/data_loader.py", str(e)))

# Teste 5: Services - Optimizer (pode falhar se ortools não estiver instalado)
try:
    from services.optimizer import OptimizerService, OptimizerConfig
    test_results.append(("✅", "services/optimizer.py", "OK"))
except ImportError as e:
    test_results.append(("⚠️", "services/optimizer.py", f"Ortools não instalado: {e}"))
except Exception as e:
    test_results.append(("❌", "services/optimizer.py", str(e)))

# Teste 6: UI Components (sintaxe apenas)
try:
    import ast
    with open('ui/components.py', 'r', encoding='utf-8') as f:
        ast.parse(f.read())
    test_results.append(("✅", "ui/components.py", "Sintaxe OK"))
except SyntaxError as e:
    test_results.append(("❌", "ui/components.py", f"Erro de sintaxe: {e}"))
except Exception as e:
    test_results.append(("⚠️", "ui/components.py", f"Streamlit não disponível (OK): {e}"))

# Teste 7: DataLoader funcional
try:
    loader = DataLoader(config.DISCIPLINAS_PATH, config.OFERTAS_PATH)
    dados = loader.carregar_dados()
    test_results.append(("✅", "DataLoader.carregar_dados()", f"OK - {len(dados.disciplinas)} disciplinas"))
except Exception as e:
    test_results.append(("❌", "DataLoader.carregar_dados()", str(e)))

# Exibir resultados
print("\n")
for status, module, message in test_results:
    print(f"{status} {module:30s} - {message}")

# Resumo
passed = sum(1 for s, _, _ in test_results if s == "✅")
warnings = sum(1 for s, _, _ in test_results if s == "⚠️")
failed = sum(1 for s, _, _ in test_results if s == "❌")

print(f"\n" + "=" * 70)
print(f"📊 RESUMO: {passed} ✅ | {warnings} ⚠️ | {failed} ❌")
print("=" * 70)

if failed > 0:
    print("\n⚠️ ATENÇÃO: Alguns testes falharam. Verifique os erros acima.")
else:
    print("\n✅ Todos os testes críticos passaram!")



🧪 TESTES DO SISTEMA


✅ config.py                      - OK
✅ models/                        - OK
✅ utils/                         - OK
✅ services/data_loader.py        - OK
✅ services/optimizer.py          - OK
✅ ui/components.py               - Sintaxe OK
✅ DataLoader.carregar_dados()    - OK - 60 disciplinas

📊 RESUMO: 7 ✅ | 0 ⚠️ | 0 ❌

✅ Todos os testes críticos passaram!


In [13]:
# Testar se app.py pode ser importado sem erros
print("\n" + "=" * 70)
print("🧪 TESTE PRÉ-INICIALIZAÇÃO")
print("=" * 70)

try:
    # Verificar se venv_python está disponível
    if 'venv_python' not in locals():
        current_dir = Path.cwd()
        venv_dir = current_dir / "venv"
        is_windows = platform.system() == "Windows"
        if is_windows:
            venv_python = venv_dir / "Scripts" / "python.exe"
        else:
            venv_python = venv_dir / "bin" / "python"
    
    # Testar importação do app.py
    print("\n📋 Testando importação de app.py...")
    result = subprocess.run(
        [str(venv_python), "-c", 
         "import sys; sys.path.insert(0, '.'); from app import *"],
        capture_output=True,
        text=True,
        timeout=10,
        cwd=str(current_dir)
    )
    
    if result.returncode == 0:
        print("✅ app.py pode ser importado sem erros!")
    else:
        print("\n❌ Erro ao importar app.py:")
        print("=" * 70)
        if result.stderr:
            print(result.stderr)
        if result.stdout:
            print(result.stdout)
        print("=" * 70)
        print("\n⚠️ Corrija os erros acima antes de iniciar o Streamlit.")
        raise RuntimeError("app.py tem erros de importação")
        
except Exception as e:
    print(f"\n❌ Erro no teste: {e}")
    import traceback
    print(traceback.format_exc())
    raise

print("\n✅ Teste pré-inicialização concluído com sucesso!")
print("   Você pode prosseguir para iniciar o Streamlit.")



🧪 TESTE PRÉ-INICIALIZAÇÃO

📋 Testando importação de app.py...
✅ app.py pode ser importado sem erros!

✅ Teste pré-inicialização concluído com sucesso!
   Você pode prosseguir para iniciar o Streamlit.


In [14]:
print("\n" + "=" * 70)
print("🚀 INICIANDO APLICAÇÃO WEB")
print("=" * 70)

# Verificar se streamlit está instalado
try:
    import streamlit
    print(f"\n✅ Streamlit instalado (versão {streamlit.__version__})")
except ImportError:
    print("\n❌ Streamlit não está instalado!")
    print("   Execute: pip install streamlit")
    raise

# Porta padrão do Streamlit
STREAMLIT_PORT = 8501
STREAMLIT_URL = f"http://localhost:{STREAMLIT_PORT}"

print(f"\n📡 URL da aplicação: {STREAMLIT_URL}")
print(f"📁 Arquivo principal: app.py")
print(f"⏰ Iniciando em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")



🚀 INICIANDO APLICAÇÃO WEB

✅ Streamlit instalado (versão 1.51.0)

📡 URL da aplicação: http://localhost:8501
📁 Arquivo principal: app.py
⏰ Iniciando em: 2025-11-29 12:02:30


In [15]:
# Iniciar Streamlit em background
import os
import socket
import urllib.request
import time
import webbrowser
import threading

# Verificar se venv_python está disponível
if 'venv_python' not in locals():
    current_dir = Path.cwd()
    venv_dir = current_dir / "venv"
    is_windows = platform.system() == "Windows"
    if is_windows:
        venv_python = venv_dir / "Scripts" / "python.exe"
    else:
        venv_python = venv_dir / "bin" / "python"
    if not venv_python.exists():
        raise FileNotFoundError(f"Venv não encontrado: {venv_python}")

print(f"   Usando Python do venv: {venv_python}")

# Verificar se já existe um processo Streamlit rodando
def check_port_open(port):
    """Verifica se uma porta está aberta e escutando."""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        result = sock.connect_ex(('localhost', port))
        sock.close()
        return result == 0
    except:
        return False

if check_port_open(STREAMLIT_PORT):
    print(f"\n⚠️ Porta {STREAMLIT_PORT} já está em uso.")
    print("   A aplicação pode já estar rodando.")
    print(f"   Acesse: {STREAMLIT_URL}")
    print(f"\n✅ Servidor já está disponível!")
    webbrowser.open(STREAMLIT_URL)
else:
    # Iniciar Streamlit usando uma abordagem que funciona completamente no notebook
    print("\n🔄 Iniciando servidor Streamlit...")
    print("   (Isso pode levar alguns segundos...)\n")
    
    # Criar arquivo de log no diretório atual
    log_file = current_dir / "streamlit.log"
    
    try:
        # Abordagem simplificada: usar subprocess.Popen diretamente
        # No Windows, usar CREATE_NO_WINDOW para não mostrar janela
        creation_flags = 0
        if is_windows:
            # CREATE_NO_WINDOW = 0x08000000
            creation_flags = 0x08000000
        
        # Abrir arquivo de log
        log_handle = open(log_file, 'w', encoding='utf-8', buffering=1)
        
        streamlit_process = subprocess.Popen(
            [str(venv_python), "-m", "streamlit", "run", "app.py",
             "--server.port", str(STREAMLIT_PORT),
             "--server.headless", "true",
             "--server.address", "localhost"],
            stdout=log_handle,
            stderr=subprocess.STDOUT,
            cwd=str(current_dir),
            creationflags=creation_flags if is_windows else 0
        )
        
        print(f"✅ Processo Streamlit iniciado (PID: {streamlit_process.pid})")
        print(f"📋 Log sendo salvo em: {log_file}")
        print(f"\n⏳ Aguardando servidor iniciar...")
        
        # Aguardar e verificar se o servidor está realmente escutando
        max_attempts = 30
        server_ready = False
        
        for attempt in range(max_attempts):
            time.sleep(1)
            
            # Verificar se o processo ainda está rodando
            if streamlit_process.poll() is not None:
                # Processo terminou, fechar log e ler erros
                log_handle.close()
                print(f"\n❌ Servidor Streamlit parou inesperadamente!")
                print(f"   Código de saída: {streamlit_process.returncode}")
                
                # Ler erros do arquivo
                if log_file.exists():
                    try:
                        with open(log_file, 'r', encoding='utf-8', errors='ignore') as f:
                            errors = f.read()
                            if errors:
                                print(f"\n📋 ERROS ENCONTRADOS:")
                                print("=" * 70)
                                # Mostrar últimos 3000 caracteres
                                error_text = errors[-3000:] if len(errors) > 3000 else errors
                                print(error_text)
                                print("=" * 70)
                    except Exception as e:
                        print(f"\n⚠️ Não foi possível ler o log: {e}")
                
                raise RuntimeError(f"Streamlit não iniciou corretamente (código: {streamlit_process.returncode})")
            
            # Verificar se a porta está aberta
            if check_port_open(STREAMLIT_PORT):
                # Tentar fazer uma requisição HTTP para confirmar
                try:
                    response = urllib.request.urlopen(STREAMLIT_URL, timeout=3)
                    if response.getcode() == 200:
                        server_ready = True
                        # Fechar handle do log (mas manter arquivo)
                        log_handle.close()
                        break
                except urllib.error.URLError:
                    # Ainda não está pronto, continuar tentando
                    pass
                except:
                    pass
            
            if attempt < max_attempts - 1:
                # Mostrar progresso
                dots = "." * ((attempt % 3) + 1)
                print(f"   Aguardando{dots} ({attempt + 1}/{max_attempts})", end='\r')
        
        if server_ready:
            print(f"\n\n✅ Servidor Streamlit está rodando e respondendo!")
            print(f"\n🌐 Abrindo navegador em: {STREAMLIT_URL}")
            
            # Abrir navegador
            try:
                webbrowser.open(STREAMLIT_URL)
                print(f"\n✅ Navegador aberto!")
            except Exception as e:
                print(f"\n⚠️ Não foi possível abrir o navegador automaticamente: {e}")
                print(f"   Por favor, acesse manualmente: {STREAMLIT_URL}")
            
            print(f"\n📝 INSTRUÇÕES:")
            print(f"   1. Use a interface web para selecionar disciplinas concluídas")
            print(f"   2. Configure o semestre de início")
            print(f"   3. Clique em 'Encontrar Grade Otimizada'")
            print(f"   4. Visualize os resultados na interface")
            print(f"\n⚠️ Para parar o servidor, execute a célula abaixo ou feche este notebook.")
            print(f"\n💡 Se o navegador não abrir automaticamente, acesse: {STREAMLIT_URL}")
            print(f"\n📋 Log completo disponível em: {log_file}")
        else:
            # Fechar handle do log
            log_handle.close()
            print(f"\n\n⚠️ Servidor iniciou mas não está respondendo ainda.")
            print(f"   Tente acessar manualmente: {STREAMLIT_URL}")
            print(f"   Ou aguarde mais alguns segundos e recarregue a página.")
            
            # Mostrar últimas linhas do log
            if log_file.exists():
                try:
                    with open(log_file, 'r', encoding='utf-8', errors='ignore') as f:
                        lines = f.readlines()
                        if lines:
                            print(f"\n📋 ÚLTIMAS LINHAS DO LOG:")
                            print("=" * 70)
                            for line in lines[-20:]:  # Últimas 20 linhas
                                if line.strip():
                                    print(line.rstrip())
                            print("=" * 70)
                except:
                    pass
            
    except Exception as e:
        # Tentar fechar handle se ainda estiver aberto
        try:
            if 'log_handle' in locals():
                log_handle.close()
        except:
            pass
            
        print(f"\n❌ Erro ao iniciar Streamlit: {e}")
        import traceback
        print(f"\n📋 Traceback completo:")
        print("=" * 70)
        print(traceback.format_exc())
        print("=" * 70)
        
        # Tentar ler erros do arquivo
        if log_file.exists():
            try:
                with open(log_file, 'r', encoding='utf-8', errors='ignore') as f:
                    errors = f.read()
                    if errors:
                        print(f"\n📋 ERROS DO STREAMLIT:")
                        print("=" * 70)
                        error_text = errors[-3000:] if len(errors) > 3000 else errors
                        print(error_text)
                        print("=" * 70)
            except:
                pass
        
        print(f"\n💡 Tente executar manualmente no terminal:")
        print(f"   {venv_python} -m streamlit run app.py")
        raise


   Usando Python do venv: d:\Arquivos\Desktop\otm\venv\Scripts\python.exe

⚠️ Porta 8501 já está em uso.
   A aplicação pode já estar rodando.
   Acesse: http://localhost:8501

✅ Servidor já está disponível!


## 7. Status e Monitoramento {#status-monitoramento}

Verificando status do servidor e exibindo informações úteis.


In [16]:
# # Verificar status do processo
# if 'streamlit_process' in locals():
#     status = streamlit_process.poll()
#     if status is None:
#         print("\n" + "=" * 70)
#         print("✅ STATUS DO SERVIDOR")
#         print("=" * 70)
#         print(f"\n🟢 Servidor Streamlit está RODANDO")
#         print(f"   PID: {streamlit_process.pid}")
#         print(f"   URL: {STREAMLIT_URL}")
#         print(f"   Hora de início: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
#         print(f"\n📊 INFORMAÇÕES DO PROJETO:")
#         print(f"   • Total de disciplinas: {len(disciplinas_data)}")
#         print(f"   • Disciplinas com oferta: {len(disciplinas_com_oferta)}")
#         print(f"   • Total de ofertas: {len(ofertas_data)}")
#         print(f"   • Créditos máximos por semestre: {config.CREDITOS_MAXIMOS_POR_SEMESTRE}")
#         print(f"   • Total de créditos do curso: {config.TOTAL_CREDITOS_CURSO}")
#         print(f"\n💡 DICAS:")
#         print(f"   • A aplicação está rodando em segundo plano")
#         print(f"   • Você pode continuar usando este notebook")
#         print(f"   • Para parar o servidor, execute a célula abaixo")
#     else:
#         print(f"\n⚠️ Servidor Streamlit parou (código de saída: {status})")
# else:
#     print("\n⚠️ Servidor não foi iniciado ainda. Execute a célula anterior.")


### Parar o Servidor

Execute a célula abaixo para parar o servidor Streamlit.


In [17]:
# # Parar servidor Streamlit
# if 'streamlit_process' in locals():
#     print("\n" + "=" * 70)
#     print("🛑 PARANDO SERVIDOR")
#     print("=" * 70)
    
#     try:
#         # Tentar terminar graciosamente
#         streamlit_process.terminate()
#         print("\n⏳ Aguardando processo terminar...")
        
#         # Aguardar até 5 segundos
#         try:
#             streamlit_process.wait(timeout=5)
#             print("✅ Servidor parado com sucesso!")
#         except subprocess.TimeoutExpired:
#             # Forçar término
#             print("⚠️ Forçando término do processo...")
#             streamlit_process.kill()
#             streamlit_process.wait()
#             print("✅ Servidor forçado a parar.")
#     except Exception as e:
#         print(f"❌ Erro ao parar servidor: {e}")
#         print("💡 Você pode precisar parar manualmente usando o gerenciador de tarefas.")
# else:
#     print("\n⚠️ Nenhum servidor em execução para parar.")


---

## 📝 Notas Finais

Este notebook fornece:

- ✅ Validação completa do ambiente e dependências
- ✅ Validação e análise dos dados
- ✅ Testes automatizados do sistema
- ✅ Inicialização automática da aplicação web
- ✅ Monitoramento do status do servidor

**Desenvolvido para:** Otimização de Grade Horária usando MILP

**Como usar:**
1. Execute todas as células em ordem
2. Aguarde o servidor Streamlit iniciar
3. Use a interface web que abrirá automaticamente
4. Execute a célula de parar servidor quando terminar

---
